# Imports and configs


In [ ]:
!pip install perlin-noise
!pip install opencv-python
!pip install tensorflow_addons
!pip3 install git+https://github.com/pvigier/perlin-numpy

In [ ]:
import gc
import os
import cv2
import random
import pathlib
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import IPython.display as display

from PIL import Image
from google.colab import drive
from keras import Model
from keras.utils import plot_model
from keras.layers import Concatenate, Average
from perlin_noise import PerlinNoise
from keras.optimizers import SGD, Adam
from keras.models import Sequential, model_from_json
from keras.callbacks import LearningRateScheduler, ModelCheckpoint,TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, Dense, Activation, Flatten, Dropout, Input, Layer
from perlin_numpy import (generate_fractal_noise_2d, generate_fractal_noise_3d,generate_perlin_noise_2d, generate_perlin_noise_3d)

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 32
CLASS_COUNT = 43

DATA_DIR = "/content/gtsrb"

log_and_model_path = '/content/drive/MyDrive/project_logs'

AUTOTUNE = tf.data.experimental.AUTOTUNE
normalize = tf.keras.layers.Rescaling(1.0/255)

In [ ]:
drive.mount('/content/drive',force_remount=True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.random.set_seed(1)

In [ ]:
!mkdir /content

In [ ]:

!unzip /content/drive/MyDrive/gtsrb.zip -d /content/



# Convert ppm to png

In [ ]:
import glob

def ppm2png(path):
    df = None
    type = path.split("\\")

    if (type[-1] == "test"):
        df = pd.read_csv(path + "/GT-final_test.test.csv", ";")
        df.set_index("Filename", inplace=True)

    for filepathdir in glob.iglob(path +"/*/"):
        if(type[-1] == "train"):
            df = pd.read_csv(filepathdir + "GT-" + filepathdir.split("\\")[-2] + ".csv", ";")
            df.set_index("Filename", inplace=True)

        for filepath in glob.iglob(filepathdir + "/*.ppm"):

            im = Image.open(filepath)
            row = df.loc[filepath.split("\\")[-1]]

            if not row.empty:
                im = im.crop((row[2],row[3],row[4],row[5]))



            filepath2 = os.path.splitext(filepath)[0] + ".png"
            im.save(filepath2)
            #os.remove(filepath)

ppm2png("gtsrb\\test")
ppm2png("gtsrb\\train")

# Auxiliary functions to display images


In [ ]:
def show_batch(cols, image_batch, label_batch):

    rows = int(BATCH_SIZE / cols)
    if rows * cols < BATCH_SIZE:
        rows += 1
    width = 3 * rows
    height = 3 * cols


    f, axes= plt.subplots(rows,cols,figsize=(height,width))
    fig=plt.figure()
    for n in range(BATCH_SIZE):

        subplot_title=("class "+ classNames[label_batch[n]==1][0])
        axes.ravel()[n].set_title(subplot_title)
        axes.ravel()[n].imshow(image_batch[n])
        axes.ravel()[n].axis('off')

    fig.tight_layout()
    plt.show()


def show_history(history):
    print(history.history.keys())

    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='lower right')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()


def show_accuracies():
    fig, ax = plt.subplots()
    X = np.arange(2)

    models = ['bad val set', 'good val set']
    plt.bar(X, [evalV1[1], evalV2[1]], width = 0.4, color = 'b', label='test')
    plt.bar(X + 0.4, [valV1[1], valV2[1]], color = 'r', width = 0.4, label = "val")
    plt.xticks(X + 0.4 / 2, models)
    plt.ylim(top = 1.0, bottom = 0.70)
    plt.legend(loc='upper left')
    plt.show()


def show_misclassified(predictions, ground_truth, images, num_rows= 5, num_cols=3):

    # Plot the first X test images with wrong predictions.
    num_images = num_rows*num_cols
    plt.figure(figsize=(2*2*num_cols, 2*num_rows))
    i = 0
    k = 0
    while k < len(images) and i < num_images:
        predicted_label = np.argmax(predictions[k])
        gt = np.where(ground_truth[k])[0][0]
        if predicted_label != gt:
            plt.subplot(num_rows, 2*num_cols, 2*i+1)
            plot_image(k, predictions[k], gt, images)
            plt.subplot(num_rows, 2*num_cols, 2*i+2)
            plot_value_array(k, predictions[k], ground_truth)
            i += 1
        k += 1
    plt.tight_layout()
    plt.show()


def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label, img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(classNames[predicted_label],
                                100*np.max(predictions_array),
                                classNames[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(43))
  plt.yticks([])
  thisplot = plt.bar(range(CLASS_COUNT), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[np.where(true_label)[0][0]].set_color('blue')

def plot_predictions(predictions, ground_truth, images, num_rows= 5, num_cols=3 ):

    num_images = num_rows*num_cols
    plt.figure(figsize=(2*2*num_cols, 2*num_rows))
    for i in range(min(num_images,len(images))):
        gt = np.where(ground_truth[i])[0][0]
        plt.subplot(num_rows, 2*num_cols, 2*i+1)
        plot_image(i, predictions[i], gt, images)
        plt.subplot(num_rows, 2*num_cols, 2*i+2)
        plot_value_array(i, predictions[i], ground_truth)
    plt.tight_layout()
    plt.show()


def show_confusion_matrix(model, dataset):

    all_labels = []
    all_preds = []

    for images , labels in dataset.take(-1):
        numpy_labels = labels.numpy()
        numpy_images = images.numpy()
        preds = model.predict(numpy_images, verbose=0)

        all_labels += [np.argmax(x) for x in numpy_labels]
        all_preds += [np.argmax(x) for x in preds]

    conf_mat = tf.math.confusion_matrix(all_labels, all_preds)

    df_cm = pd.DataFrame(conf_mat.numpy(), range(CLASS_COUNT), range(CLASS_COUNT))
    plt.figure(figsize=(15,10))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='d') # font size

    plt.show()

    res_correct = {}
    res_incorrect = {}
    for i in range(CLASS_COUNT):
      res_correct[i] = 0
      res_incorrect[i] = 0

    for i in range(len(all_preds)):
        if all_preds[i] == all_labels[i]:
            res_correct[all_labels[i]] += 1
        else:
            res_incorrect[all_labels[i]] += 1

    for i in range(len(res_correct)):

        print('class: ', i, ' total images: ', res_correct[i] + res_incorrect[i],' % correct: ', res_correct[i] / (res_correct[i] + res_incorrect[i]))

# Auxiliary functions to load images

In [ ]:
classNames = np.array(['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009',
                       '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019',
                       '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029',
                       '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039',
                       '00040', '00041', '00042'])


def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory

  return parts[-2] == classNames


def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.

  # resize the image to the desired size.
  im_shape = img.shape
  img = tf.py_function(call_change_contrast, [img], tf.uint8)
  img.set_shape(im_shape)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, [IMAGE_SIZE,IMAGE_SIZE])
  return img # tensorflow resize changes dtype to float32 and it doesn't make sense in rgb format


def get_bytes_and_label(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label


class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()


def prepareCallbacks(path):
    file_path = f'{log_and_model_path}/{path}/cp.ckpt'

    checkpointer = ModelCheckpoint(filepath= file_path,
                                monitor = 'val_accuracy',
                                verbose=1,
                                save_weights_only=True,
                                save_best_only=True)


    earlyStopper = EarlyStopping(monitor='val_accuracy', min_delta = 0.0001, patience = 15, verbose = 1)
    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000000001, verbose = 1)

    tbCallBack = TensorBoard(log_dir=f'{log_and_model_path}/{path}_log', histogram_freq=0, write_graph=True, write_images=True)

    return file_path, [checkpointer, earlyStopper, reduceLR, tbCallBack, MyCustomCallback()]

# Auxiliary functions for data augmentation

In [ ]:
def process_image(image, label):

    # random rotate 5 degrees
    r = tf.random.uniform(shape=(), minval=-0.175, maxval=0.175, dtype=tf.dtypes.float32)
    image = tfa.image.rotate(image, r)

    # translate image up to 10%
    rx = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)
    ry = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)
    image = tfa.image.translate(image, [rx, ry])

    # change hue, saturation and value
    image = tfa.image.random_hsv_in_yiq(image, 0.2, 0.4, 1.1, 0.4, 1.1)
    image = tf.clip_by_value(image,0,255)

    return image, label


def call_change_contrast(img):
    with tf.device("/gpu:0"):
      lab= cv2.cvtColor(img.numpy(), cv2.COLOR_RGB2LAB)
      l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
      clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
      cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
      limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
      enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
      enhanced_img = tf.convert_to_tensor(enhanced_img, dtype=tf.uint8)
    return enhanced_img


def change_contrast(image,label):
    im_shape = image.shape
    imagen = tf.py_function(call_change_contrast, [image], tf.uint8)
    imagen.set_shape(im_shape)
    return imagen, label


def change_sharpness(img,label):
    img = tfa.image.sharpness(img,0.2)
    return img,label


def process_contrast(img,label):
  with tf.device("/gpu:0"):
    contrast = tf.keras.layers.RandomContrast(factor=(0.3,1.3),value_range=(0,1))
    img = tf.cast(contrast(img,training=True),tf.float32)
    img = tf.clip_by_value(img,0,1)
  return img,label


def process_hue(img,label):
  with tf.device("/gpu:0"):
    hue = tf.image.random_hue(img,0.3)
    img = tf.cast(hue,tf.float32)
    img = tf.clip_by_value(img,0,1)
  return img,label


def process_brightness(img,label):
  with tf.device("/gpu:0"):
    bright = tf.keras.layers.RandomBrightness(factor=(-0.8,0.8),value_range=(0,1))
    img =  bright(img,training=True)
    img = tf.clip_by_value(img,0,1)
  return img,label


def process_saturation(img,label):
  with tf.device("/gpu:0"):
    img = tf.image.random_saturation(img,0.5,3)
    img = tf.clip_by_value(img,0,1)
  return img,label


def process_rotation(img,label):
    with tf.device("/gpu:0"):
      img = tfa.image.rotate(img, tf.random.uniform(shape=(), minval=-0.5 , maxval=0.5))
      image = tf.clip_by_value(img,0,1)
    return image, label


def process_zoom(img,label):
    with tf.device("/gpu:0"):
      rand_zoom = tf.keras.layers.RandomZoom(height_factor=(-0.15,0))
      img = rand_zoom(img,training=True)
      img = tf.cast(img,tf.float32)
      img = tf.clip_by_value(img,0,1)
    return img,label


def perlin_noise(img):
  with tf.device("/gpu:0"):
    noise = generate_perlin_noise_2d((IMAGE_SIZE, IMAGE_SIZE), (16, 16))
    noise = cv2.cvtColor(np.uint8(np.round(np.interp(noise,(noise.min(),noise.max()),(0, 255)))),cv2.COLOR_GRAY2BGR)

    dst = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3) , np.uint8 )
    img = tf.image.convert_image_dtype(img, tf.uint8)

    cv2.addWeighted (img.numpy(), 0.5, noise, 0.5, 0, dst)
    img = tf.image.convert_image_dtype(dst, tf.float32)

  return img


def apply_perlin(image,label):
    im_shape = image.shape
    imagen = tf.py_function(perlin_noise, [image], tf.float32)
    imagen.set_shape(im_shape)
    img = tf.clip_by_value(imagen,0,1)
    return img, label


def motion_blur(image):
  with tf.device("/gpu:0"):
    kernel_size = 8

# Create the vertical kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))

# Create a copy of the same for creating the horizontal kernel.
    kernel_h = np.copy(kernel_v)

# Fill the middle row with ones.
  #kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size)

# Normalize.
#kernel_v /= kernel_size
    kernel_h /= kernel_size

# Apply the vertical kernel.
    img = cv2.filter2D(image.numpy(), -1, kernel_h)
    img = tf.cast(img,tf.float32)
  return img


def apply_blur(image,label):
  im_shape = image.shape
  imagen = tf.py_function(motion_blur, [image], tf.float32)
  imagen.set_shape(im_shape)
  img = tf.clip_by_value(imagen,0,1)
  return img, label


def process_translate(img,label):
  with tf.device("/gpu:0"):
  # translate image up to 10%
    tr = tf.keras.layers.RandomTranslation(height_factor=(-0.3,0.3),width_factor=(-0.3,0.3),fill_mode='constant')
    image = tf.cast(tr(img,training=True),tf.float32)
    img = tf.clip_by_value(image,0,1)
  return img,label


def process_shear(image, label):
  with tf.device("/gpu:0"):
    image = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.2, maxval=0.2))
    sx = tf.random.uniform(shape=(), minval=-0.15, maxval=0.15, dtype=tf.dtypes.float32)
    image = tfa.image.transform(image, [1, sx, -sx*32,   0,1,0,  0,0])
    image = tf.clip_by_value(image,0,1)
    return image, label


def process_crop(image, label):
  with tf.device("/gpu:0"):
    c = tf.random.uniform(shape=(), minval=24, maxval=32, dtype=tf.dtypes.float32)
    image = tf.image.random_crop(image, size=[c,c,3])
    image = tf.image.resize(image ,size= [32,32])
    image = tf.cast(image,tf.float32)
    image = tf.clip_by_value(image,0,1)
  return image, label


def augment_blur(img,label):
  img = tfa.image.gaussian_filter2d(img)
  return img,label


def call_gaussian_laplace(img):
  # gray = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2GRAY)
  gaussian = cv2.GaussianBlur(img.numpy(),(11,11),0)
  laplace = cv2.Laplacian(gaussian, cv2.CV_64F, ksize=3)
  laplace_abs = np.uint(np.absolute(laplace))
  return laplace_abs


def gaussian_laplace(image,label):
    im_shape = image.shape
    imagen = tf.py_function(call_gaussian_laplace, [image], tf.uint8)
    imagen.set_shape(im_shape)
    return imagen, label


def perspective_transformation(img):
  with tf.device("/gpu:0"):
    y,x = img.shape[:2]
    src = np.float32([[0,0], [x,0], [x,y], [0,y]])

    # source transformation vertices
    h = [0, 0, 0, 0]
    w = [0, 0, 0, 0]

    # choose a random point
    v = random.randint(0, 3)
    if v == 0 or v == 2:
        mult = [-1, 1]
    else :
        mult = [1, -1]

    for i in range (0 , 2) :
        # determine new y - coordinate
        h[( v + i ) % 4] = round(y * random.uniform(0.05, 0.15))
        # determine new x - coordinate
        w[( v + i ) % 4] = round(x * random.uniform(0.05, 0.15))

    # transforms opposite vertices
    h[( v + 3 - i ) % 4] = mult [1] * h [( v + i ) % 4]
    w[( v + 3 - i ) % 4] = mult [0] * w [( v + i ) % 4]

    dst = np.float32([[w[0], h[0]], [x - w[1], h[1]], [x - w[2], y - h[2]] , [w[3], y - h[3]]])
    M = cv2.getPerspectiveTransform(src, dst)
  return cv2.warpPerspective(img.numpy(), M, (x,y))


def process_perspective(image,label):
  im_shape = image.shape
  imagen = tf.py_function(perspective_transformation, [image], tf.float32)
  imagen.set_shape(im_shape)
  img = tf.clip_by_value(imagen,0,1)
  return img, label


def eraser(input_img):
      s_l=0.02
      s_h=0.4
      r_1=0.3
      r_2=1/0.3
      v_l=0
      v_h=1
      pixel_level = True
      with tf.device("/gpu:0"):
        input_img = input_img.numpy()
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img


def process_erasing(image,label):
  # load image to img
  im_shape = image.shape
  imagen = tf.py_function(eraser, [image], tf.float32)
  imagen.set_shape(im_shape)
  img = tf.clip_by_value(imagen,0,1)
  return img, label


def confetti_noise(image, kernel=(0.03, 0.03), probability=0.03, spacing=0.015):
  h,w = image.shape[:2]
  kh,kw = round(kernel[0] * h), round(kernel[1] * w)
  s = round(spacing * min(kernel[0], kernel[1]))
  y,x = 0,0

  new_image = image.numpy()

  for i in range(h - kh):
    if y > 0:
      y -= 1
      continue

    for j in range(w - kw + 1):
      if x > 0:
        x -= 1
        continue

      if random.random() <= probability:
        # insert colored block
        new_image[i : i + kh, j : j + kw, :3] = random.uniform(0,1), random.uniform(0,1), random.uniform(0,1)
        y,x = kh + s, kw + s

  return new_image


def process_confetti_noise(image,label):
  im_shape = image.shape
  imagen = tf.py_function(confetti_noise, [image], tf.float32)
  imagen = tf.py_function(perlin_noise, [imagen], tf.float32)
  imagen.set_shape(im_shape)
  img = tf.clip_by_value(imagen,0,1)
  return img, label


def process_hlines(image, label):
    image = tf.expand_dims(image, axis=0)
    size_ver = int(tf.random.uniform(shape=(), minval=0, maxval=8, dtype=tf.dtypes.int32)/2)
    size_ver = tf.cond(tf.math.equal(size_ver % 2, 1),
                        lambda: size_ver + 1,
                        lambda: size_ver)
    
    val = tf.random.uniform(shape=(), minval=0, maxval=1, dtype=tf.dtypes.float32)
    return tf.squeeze(tfa.image.random_cutout(image, (size_ver, IMAGE_SIZE), constant_values = val), axis=0), label

def process_vlines(image, label):
    image = tf.expand_dims(image, axis=0)
    size_hor = int(tf.random.uniform(shape=(), minval=0, maxval=8, dtype=tf.dtypes.int32)/2)
    size_hor = tf.cond(tf.math.equal(size_hor % 2, 1),
                        lambda: size_hor + 1,
                        lambda: size_hor)
    
    val = tf.random.uniform(shape=(), minval=0, maxval=1, dtype=tf.dtypes.float32)
    return tf.squeeze(tfa.image.random_cutout(image, (IMAGE_SIZE, size_hor), constant_values = val), axis=0), label
  

def process_image_testing(img,l):
  img,l = process_translate(img,l)
  img,l = process_rotation(img,l)
  img,l = process_shear(img,l)
  img,l = process_hue(img,l)
  img,l = process_saturation(img,l)
  img,l = process_brightness(img,l)
  return img,l

# Loading the images

In [ ]:
all_train_listset = tf.data.Dataset.list_files(f'{DATA_DIR}/train/*/*.png')
all_train_set = all_train_listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)

all_train_set = all_train_set.shuffle(buffer_size=39210)
train_set = all_train_set.take(int(0.8 * all_train_set.cardinality().numpy()))
val_set = all_train_set.skip(int(0.8 * all_train_set.cardinality().numpy()))

test_listset = tf.data.Dataset.list_files(f'{DATA_DIR}/test/*/*.png')
test_set = test_listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)

print('All train set size:', all_train_set.cardinality().numpy())
print('... train set size:', train_set.cardinality().numpy())
print('...   val set size:', val_set.cardinality().numpy())
print('...  test set size:', test_set.cardinality().numpy())

## Printing Some Images and dataset length

In [ ]:
for image, label in train_set.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  plt.imshow(image)
  plt.show()

# Test Transformations

In [ ]:
for img, label in train_set.take(10):
  # print("Image shape: ", img.numpy().shape)
  # print("Label: ", label.numpy())
  # plt.imshow(img)
  # plt.show()

  # gray = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2GRAY)
  # imgT = cv2.GaussianBlur(gray,(11,11),0)
  # laplacian1 = cv2.Laplacian(imgT, cv2.CV_64F, ksize=3)
  # laplacian1_abs = np.uint(np.absolute(laplacian1))

  # img = tfa.image.translate(img.numpy(), [random.randint(-10,10),random.randint(-10,10)])

  # c = tf.random.uniform(shape=(), minval=24, maxval=32, dtype=tf.dtypes.float32)
  # img = tf.image.random_crop(img.numpy(), size=[c,c,3])
  # img = tf.image.resize(img ,size= [48,48])
  # img = tf.cast(img,tf.uint8)

  # img,label = apply_blur(img,label)

  # img,label = process_image_testing(img,label)

  # img = confetti_noise(img)

  # img,label = apply_perlin(img,label)

  img2,_ = process_erasing(img,label)

  # plt.imshow(img)
  # plt.imshow(laplacian1_abs, cmap='gray')
  # plt.show()

  _, axarr = plt.subplots(1,2)
  axarr[0].imshow(img)
  axarr[1].imshow(img2)

# Model I: Using only Convolutional layers

In [ ]:
def model_I(classCount, imgSize, channels):
    model = Sequential()

    model.add(Input(shape=(imgSize, imgSize, channels)))

    model.add(Conv2D(64, (5, 5)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2D(64, (5, 5) ))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (5, 5) ) )
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.2))

    model.add(Dense(classCount, activation='softmax'))


    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    return model

# Model II - Just changing model parameters (more filters)

In [ ]:
def model_II(classCount, imgSize, channels):
    model = Sequential()

    model.add(Conv2D(128, (5, 5), input_shape=(imgSize, imgSize, channels)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2D(128, (5, 5) ))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (5, 5) ) )
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.2))

    model.add(Dense(classCount, activation='softmax'))


    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    return model

# Model III

In [ ]:
def model_III(classCount, imgSize, channels):

    model = Sequential()

    model.add(Conv2D(100, (5, 5), input_shape=(imgSize, imgSize, channels)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Conv2D(150, (5, 5) ))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))


    model.add(Conv2D(250, (5, 5) ))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))


    model.add(Flatten())
    model.add(Dense(350))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.2))

    model.add(Dense(classCount, activation='softmax'))


    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    return model

# Dataset 1: no modifications

In [ ]:
train_set_1 = train_set
train_set_1 = train_set_1.cache()
train_set_1 = train_set_1.batch(batch_size = BATCH_SIZE)
train_set_1 = train_set_1.prefetch(buffer_size = AUTOTUNE)

val_set_1 = val_set
val_set_1 = val_set_1.cache()
val_set_1 = val_set_1.batch(batch_size = BATCH_SIZE)
val_set_1 = val_set_1.prefetch(buffer_size = AUTOTUNE)

test_set_1 = test_set
test_set_1 = test_set_1.batch(batch_size = BATCH_SIZE)

# Teste 1: model 1 dataset 1

In [ ]:
model_1_1 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_1, callbacks_1_1 = prepareCallbacks('best_1_1')

history_1_1 = model_1_1.fit(train_set_1,
                            epochs=50,
                            validation_data = val_set_1,
                            callbacks=callbacks_1_1,
                            verbose=2)

In [ ]:
show_history(history_1_1)

In [ ]:
model_1_1.load_weights(file_path_1_1)
eval_1_1 = model_1_1.evaluate(test_set_1, verbose=2)
print(eval_1_1)

In [ ]:
show_confusion_matrix(model_1_1, test_set_1)

In [ ]:
for images, labels in test_set_1.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_1_1.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Teste 2: model 2 dataset 1

In [ ]:
model_2_1 = model_II(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_2_1, callbacks_2_1 = prepareCallbacks('best_2_1')

history_2_1 = model_2_1.fit(train_set_1,
                            epochs=50,
                            validation_data = val_set_1,
                            callbacks=callbacks_2_1,
                            verbose=2)

In [ ]:
show_history(history_2_1)

In [ ]:
model_2_1.load_weights(file_path_2_1)
eval_2_1 = model_2_1.evaluate(test_set_1, verbose=2)
print(eval_2_1)

val_2_1 = model_2_1.evaluate(val_set_1, verbose=2)

In [ ]:
show_confusion_matrix(model_2_1, test_set_1)

In [ ]:
for images, labels in test_set_1.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_2_1.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)



# Teste 3: model 3 dataset 1

In [ ]:
model_3_1 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_3_1, callbacks_3_1 = prepareCallbacks('best_3_1')

history_3_1 = model_3_1.fit(train_set_1,
                            epochs=50,
                            validation_data = val_set_1,
                            callbacks=callbacks_3_1,
                            verbose=2)

In [ ]:
model_3_1.load_weights(file_path_3_1)
eval_3_1 = model_3_1.evaluate(test_set_1, verbose=2)
print(eval_3_1)

val_3_1 = model_3_1.evaluate(val_set_1, verbose=2)

# Dataset 2: processing images (rotation translate hue saturation value)

In [ ]:
train_set_2 = train_set
train_set_2 = train_set_2.cache()
train_set_2 = train_set_2.map(process_image, num_parallel_calls = AUTOTUNE)
train_set_2 = train_set_2.shuffle(buffer_size=30000)
train_set_2 = train_set_2.batch(batch_size=BATCH_SIZE)
train_set_2 = train_set_2.prefetch(buffer_size = AUTOTUNE)

val_set_2 = val_set
val_set_2 = val_set_2.cache()
val_set_2 = val_set_2.batch(batch_size = BATCH_SIZE)
val_set_2 = val_set_2.prefetch(buffer_size = AUTOTUNE)

test_set_2 = test_set
test_set_2 = test_set_2.batch(batch_size = BATCH_SIZE)

# Teste 4: model 1 dataset 2

In [ ]:
model_1_2 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_2, callbacks_1_2 = prepareCallbacks('best_1_2')

history_1_2 = model_1_2.fit(train_set_2,
                            epochs = 30,
                            validation_data = val_set_2,
                            callbacks = callbacks_1_2,
                            verbose=2)

In [ ]:
model_1_2.load_weights(file_path_1_2)
eval_1_2 = model_1_2.evaluate(test_set_2, verbose=2)
print(eval_1_2)

valV1DA = model_1_2.evaluate(val_set_2, verbose=2)

In [ ]:
for images, labels in test_set_2.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_1_2.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Teste 5: model 2 dataset 2

In [ ]:
model_2_2 = model_II(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_2_2, callbacks_2_2 = prepareCallbacks('best_2_2')

history_2_2 = model_2_2.fit(train_set_2,
                            epochs = 40,
                            validation_data = val_set_2,
                            callbacks = callbacks_2_2,
                            verbose=2)

In [ ]:
model_2_2.load_weights(file_path_2_2)
eval_2_2 = model_2_2.evaluate(test_set_2, verbose=2)
print(eval_2_2)

val_2_2 = model_2_2.evaluate(val_set_2, verbose=2)

In [ ]:
for images, labels in test_set_2.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_2_2.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 3: pre-processing (contrast)

In [ ]:
train_set_3 = train_set
train_set_3 = train_set_3.cache()
train_set_3 = train_set_3.map(change_contrast, num_parallel_calls = AUTOTUNE)
train_set_3 = train_set_3.map(process_image, num_parallel_calls = AUTOTUNE)
train_set_3 = train_set_3.shuffle(buffer_size=30000)
train_set_3 = train_set_3.batch(batch_size=BATCH_SIZE)
train_set_3 = train_set_3.prefetch(buffer_size = AUTOTUNE)

val_set_3 = val_set
val_set_3 = val_set_3.cache()
val_set_3 = val_set_3.map(change_contrast, num_parallel_calls = AUTOTUNE)
val_set_3 = val_set_3.batch(batch_size = BATCH_SIZE)
val_set_3 = val_set_3.prefetch(buffer_size = AUTOTUNE)

test_set_3 = test_set
test_set_3 = test_set_3.map(change_contrast, num_parallel_calls = AUTOTUNE)
test_set_3 = test_set_3.batch(batch_size = BATCH_SIZE)

# Teste 6: model 1 dataset 3

In [ ]:
model_1_3 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_3, callbacks_1_3 = prepareCallbacks('best_1_3')

history_1_3 = model_1_3.fit(train_set_3,
                            epochs = 35,
                            validation_data = val_set_3,
                            callbacks = callbacks_1_3,
                            verbose=2)

In [ ]:
model_1_3.load_weights(file_path_1_3)
eval_1_3 = model_1_3.evaluate(test_set_3, verbose=2)
print(eval_1_3)

valV1PP = model_1_3.evaluate(val_set_3, verbose=2)

In [ ]:
for images, labels in test_set_3.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_1_3.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Teste 7: model 2 dataset 3

In [ ]:
model_2_3 = model_II(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_2_3, callbacks_2_3 = prepareCallbacks('best_2_3')

history_2_3 = model_2_3.fit(train_set_3,
                            epochs = 50,
                            validation_data = val_set_3,
                            callbacks = callbacks_2_3,
                            verbose=2)

In [ ]:
model_2_3.load_weights(file_path_2_3)
eval_2_3 = model_2_3.evaluate(test_set_3, verbose=2)
print(eval_2_3)

val_2_3 = model_2_3.evaluate(val_set_3,verbose=2)

# Dataset 4: dynamic - color transformations

In [ ]:
train_set_4 = train_set
train_set_4 = train_set_4.cache()
train_set_4 = train_set_4.map(process_hue, num_parallel_calls = AUTOTUNE)
train_set_4 = train_set_4.map(process_saturation, num_parallel_calls = AUTOTUNE)
train_set_4 = train_set_4.map(process_brightness, num_parallel_calls = AUTOTUNE)
train_set_4 = train_set_4.shuffle(buffer_size=30000)
train_set_4 = train_set_4.batch(batch_size=BATCH_SIZE)
train_set_4 = train_set_4.prefetch(buffer_size = AUTOTUNE)

val_set_4 = val_set
val_set_4 = val_set_4.cache()
val_set_4 = val_set_4.batch(batch_size = BATCH_SIZE)
val_set_4 = val_set_4.prefetch(buffer_size = AUTOTUNE)

test_set_4 = test_set
test_set_4 = test_set_4.batch(batch_size = BATCH_SIZE)

# Teste 8: model 1 dataset 4

In [ ]:
model_1_4 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_4, callbacks_1_4 = prepareCallbacks('best_1_4')

history_1_4 = model_1_4.fit(train_set_4,
                            epochs = 50,
                            validation_data = val_set_4,
                            callbacks = callbacks_1_4,
                            verbose=2)

In [ ]:
model_1_4.load_weights(file_path_1_4)
eval_1_4 = model_1_4.evaluate(test_set_4, verbose=2)
print(eval_1_4)

val_1_4 = model_1_4.evaluate(val_set_4, verbose=2)

# Teste 9: model 2 dataset 4

In [ ]:
model_2_4 = model_II(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_2_4, callbacks_2_4 = prepareCallbacks('best_2_4')

history_2_4 = model_2_4.fit(train_set_4,
                            epochs = 50,
                            validation_data = val_set_4,
                            callbacks = callbacks_2_4,
                            verbose=2)

In [ ]:
model_2_4.load_weights(file_path_2_4)
eval_2_4 = model_2_4.evaluate(test_set_4, verbose=2)
print(eval_2_4)

val_2_4 = model_2_4.evaluate(val_set_4, verbose=2)

# Teste 10: model 3 dataset 4

In [ ]:
model_3_4 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_3_4, callbacks_3_4 = prepareCallbacks('best_3_4')

history_3_4 = model_3_4.fit(train_set_4,
                            epochs = 50,
                            validation_data = val_set_4,
                            callbacks = callbacks_3_4,
                            verbose=2)

In [ ]:
model_3_4.load_weights(file_path_3_4)
eval_3_4 = model_3_4.evaluate(test_set_4, verbose=2)
print(eval_3_4)

val_3_4 = model_3_4.evaluate(val_set_4, verbose=2)

# Dataset 5: data augmentation - geometric transformations

In [ ]:
train_set_5 = train_set
train_set_5 = train_set_5.concatenate(train_set.map(process_hue, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_saturation, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_brightness, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_perspective, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_shear, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.concatenate(train_set.map(process_crop, num_parallel_calls = AUTOTUNE))
train_set_5 = train_set_5.cache()
train_set_5 = train_set_5.shuffle(buffer_size=30000)
train_set_5 = train_set_5.batch(batch_size=BATCH_SIZE)
train_set_5 = train_set_5.prefetch(buffer_size = AUTOTUNE)

val_set_5 = val_set
val_set_5 = val_set_5.cache()
val_set_5 = val_set_5.batch(batch_size = BATCH_SIZE)
val_set_5 = val_set_5.prefetch(buffer_size = AUTOTUNE)

test_set_5 = test_set
test_set_5 = test_set_5.batch(batch_size = BATCH_SIZE)


# Teste 11: model 1 dataset 5

In [ ]:
model_1_5 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_5, callbacks_1_5 = prepareCallbacks('best_1_5')

history_1_5 = model_1_5.fit(train_set_5,
                            epochs = 50,
                            validation_data = val_set_5,
                            callbacks = callbacks_1_5,
                            verbose=2)

In [ ]:
model_1_5.load_weights(file_path_1_5)
eval_1_5 = model_1_5.evaluate(test_set_5, verbose=2)
print(eval_1_5)

val_1_5 = model_1_5.evaluate(val_set_5, verbose=2)

In [ ]:
for images, labels in test_set_5.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_1_5.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Teste 12: model 2 dataset 5

In [ ]:
model_2_5 = model_II(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_2_5, callbacks_2_5 = prepareCallbacks('best_2_5')

history_2_5 = model_2_5.fit(train_set_5,
                            epochs = 50,
                            validation_data = val_set_5,
                            callbacks = callbacks_2_5,
                            verbose=2)

In [ ]:
model_2_5.load_weights(file_path_2_5)
eval_2_5 = model_2_5.evaluate(test_set_5, verbose=2)
print(eval_2_5)

val_2_5 = model_2_5.evaluate(val_set_5, verbose=2)

# Teste 13: model 3 dataset 5

In [ ]:
model_3_5 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_3_5, callbacks_3_5 = prepareCallbacks('best_3_5')

history_3_5 = model_3_5.fit(train_set_5,
                            epochs = 50,
                            validation_data = val_set_5,
                            callbacks = callbacks_3_5,
                            verbose=2)

In [ ]:
model_3_5.load_weights(file_path_3_5)
eval_3_5 = model_3_5.evaluate(test_set_5, verbose=2)
print(eval_3_5)

val_3_5 = model_3_5.evaluate(val_set_5, verbose=2)

# Dataset 6: data augmentation - disturbance transformations

In [ ]:
train_set_6 = train_set
train_set_6 = train_set_6.concatenate(train_set.map(process_hue, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_saturation, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_brightness, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_perspective, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_shear, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(process_crop, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_6 = train_set_6.cache()
train_set_6 = train_set_6.shuffle(buffer_size=30000)
train_set_6 = train_set_6.batch(batch_size=BATCH_SIZE)
train_set_6 = train_set_6.prefetch(buffer_size = AUTOTUNE)

val_set_6 = val_set
val_set_6 = val_set_6.cache()
val_set_6 = val_set_6.batch(batch_size = BATCH_SIZE)
val_set_6 = val_set_6.prefetch(buffer_size = AUTOTUNE)

test_set_6 = test_set
test_set_6 = test_set_6.batch(batch_size = BATCH_SIZE)

# Teste 14: model 3 dataset 6

In [ ]:
model_3_6 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_3_6, callbacks_3_6 = prepareCallbacks('best_3_6')

history_3_6 = model_3_6.fit(train_set_6,
                            epochs = 50,
                            validation_data = val_set_6,
                            callbacks = callbacks_3_6,
                            verbose=2)

In [ ]:
model_3_6.load_weights(file_path_3_6)
eval_3_6 = model_3_6.evaluate(test_set_6, verbose=2)
print(eval_3_6)

val_3_6 = model_3_6.evaluate(val_set_6, verbose=2)

In [ ]:
for images, labels in test_set_6.take(-1):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_3_6.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 7: data augmentation - oclusion transformations

In [ ]:
train_set_7 = train_set
train_set_7 = train_set_7.concatenate(train_set.map(process_perspective, num_parallel_calls = AUTOTUNE))
train_set_7 = train_set_7.concatenate(train_set.map(process_crop, num_parallel_calls = AUTOTUNE))
train_set_7 = train_set_7.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_7 = train_set_7.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_7 = train_set_7.concatenate(train_set.map(process_erasing, num_parallel_calls = AUTOTUNE))
train_set_7 = train_set_7.cache()
train_set_7 = train_set_7.map(process_image_testing, num_parallel_calls = AUTOTUNE)
train_set_7 = train_set_7.shuffle(buffer_size=30000)
train_set_7 = train_set_7.batch(batch_size=BATCH_SIZE)
train_set_7 = train_set_7.prefetch(buffer_size = AUTOTUNE)

val_set_7 = val_set
val_set_7 = val_set_7.cache()
val_set_7 = val_set_7.batch(batch_size = BATCH_SIZE)
val_set_7 = val_set_7.prefetch(buffer_size = AUTOTUNE)

test_set_7 = test_set
test_set_7 = test_set_7.batch(batch_size = BATCH_SIZE)

# Teste 15: model 3 dataset 7

In [ ]:
model_3_7 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_3_7, callbacks_3_7 = prepareCallbacks('best_3_7')

history_3_7 = model_3_7.fit(train_set_7,
                            epochs = 50,
                            validation_data = val_set_7,
                            callbacks = callbacks_3_7,
                            verbose=2)

In [ ]:
model_3_7.load_weights(file_path_3_7)
eval_3_7 = model_3_7.evaluate(test_set_7, verbose=2)
print(eval_3_7)

val_3_7 = model_3_7.evaluate(val_set_7,verbose=2)

# Dataset 8: data augmentation - add gaussian blur

In [ ]:
train_set_8 = train_set
train_set_8 = train_set_8.concatenate(train_set.map(process_brightness, num_parallel_calls = AUTOTUNE))
train_set_8 = train_set_8.concatenate(train_set.map(process_contrast, num_parallel_calls = AUTOTUNE))
train_set_8 = train_set_8.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_8 = train_set_8.concatenate(train_set.map(process_zoom, num_parallel_calls = AUTOTUNE))
train_set_8 = train_set_8.concatenate(train_set.map(augment_blur, num_parallel_calls = AUTOTUNE))

lenght = train_set_8.cardinality().numpy()

train_set_8 = train_set_8.cache()
train_set_8 = train_set_8.shuffle(buffer_size=lenght)
train_set_8 = train_set_8.batch(batch_size=BATCH_SIZE)
train_set_8 = train_set_8.prefetch(buffer_size = AUTOTUNE)

val_set_8 = val_set
val_set_8 = val_set_8.cache()
val_set_8 = val_set_8.batch(batch_size = BATCH_SIZE)
val_set_8 = val_set_8.prefetch(buffer_size = AUTOTUNE)

test_set_8 = test_set
test_set_8 = test_set_8.batch(batch_size = BATCH_SIZE)

# Teste 16: model 1 dataset 8

In [ ]:
model_1_8 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_8, callbacks_1_8 = prepareCallbacks('best_1_8')

history_1_8 = model_1_8.fit(train_set_8,
                            epochs = 50,
                            validation_data = val_set_8,
                            callbacks = callbacks_1_8,
                            verbose=2)


In [ ]:
model_1_8.load_weights(file_path_1_8)
eval_1_8 = model_1_8.evaluate(test_set_8, verbose=2)
print(eval_1_8)

val_1_8 = model_1_8.evaluate(val_set_8,verbose=2)

In [ ]:
for images, labels in test_set_8.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_1_8.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 9: data augmentation - add contornos

In [ ]:
train_set_9 = train_set
# train_set_9 = train_set_9.concatenate(train_set.map(process_brightness, num_parallel_calls = AUTOTUNE))
# train_set_9 = train_set_9.concatenate(train_set.map(process_contrast, num_parallel_calls = AUTOTUNE))
# train_set_9 = train_set_9.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
# train_set_9 = train_set_9.concatenate(train_set.map(process_zoom, num_parallel_calls = AUTOTUNE))
train_set_9 = train_set_9.concatenate(train_set.map(gaussian_laplace, num_parallel_calls = AUTOTUNE))
train_set_9 = train_set_9.cache()
train_set_9 = train_set_9.shuffle(buffer_size=30000)
train_set_9 = train_set_9.batch(batch_size=BATCH_SIZE)
train_set_9 = train_set_9.prefetch(buffer_size = AUTOTUNE)

val_set_9 = val_set
val_set_9 = val_set_9.batch(batch_size = BATCH_SIZE)
val_set_9 = val_set_9.prefetch(buffer_size = AUTOTUNE)

test_set_9 = test_set
test_set_9 = test_set_9.batch(batch_size = BATCH_SIZE)

# Teste 17: model 1 dataset 9

In [ ]:
model_1_9 = model_I(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_1_9, callbacks_1_9 = prepareCallbacks('best_1_9')

history_1_9 = model_1_9.fit(train_set_9,
                            epochs = 30,
                            validation_data = val_set_9,
                            callbacks = callbacks_1_9,
                            verbose=2)

In [ ]:
model_1_9.load_weights(file_path_1_9)
eval_1_9 = model_1_9.evaluate(test_set_9, verbose=2)
print(eval_1_9)

val_1_9 = model_1_9.evaluate(val_set_9, verbose=2)

In [ ]:
for images, labels in test_set_9.take(10):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_1_9.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 10: concat best transformations and dynamic augmentation
-using only model3 since it was the one who get the best results until now

In [ ]:
train_set_10 = train_set
train_set_10 = train_set_10.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_10 = train_set_10.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_10 = train_set_10.concatenate(train_set.map(process_perspective,num_parallel_calls=AUTOTUNE))
train_set_10 = train_set_10.concatenate(train_set.map(process_crop,num_parallel_calls=AUTOTUNE))
train_set_10 = train_set_10.concatenate(train_set.map(process_erasing,num_parallel_calls=AUTOTUNE))
train_set_10 = train_set_10.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_10 = train_set_10.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_10 = train_set_10.cache()
train_set_10 = train_set_10.map(process_image_testing, num_parallel_calls = AUTOTUNE)
train_set_10 = train_set_10.shuffle(buffer_size=30000)
train_set_10 = train_set_10.batch(batch_size=BATCH_SIZE)
train_set_10 = train_set_10.prefetch(buffer_size = AUTOTUNE)

val_set_10 = val_set
val_set_10 = val_set_10.cache()
val_set_10 = val_set_10.batch(batch_size = BATCH_SIZE)
val_set_10 = val_set_10.prefetch(buffer_size = AUTOTUNE)

test_set_10 = test_set
test_set_10 = test_set_10.batch(batch_size = BATCH_SIZE)

# Teste 18: model 3 dataset 10 -> Accuracy: 99.44

In [ ]:
model_3_10 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
file_path_3_10, callbacks_3_10 = prepareCallbacks('best_3_12')
history_3_10 = model_3_10.fit(train_set_10,
                              epochs = 50,
                              validation_data = val_set_10,
                              callbacks = callbacks_3_10,
                              verbose=2)

In [ ]:
model_3_10.load_weights(file_path_3_10)
eval_3_10 = model_3_10.evaluate(test_set_10, verbose=2)
print(eval_3_10)

val_3_10 = model_3_10.evaluate(val_set_10, verbose=2)

In [ ]:
for images, labels in test_set_10.take(-1):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_3_10.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 11: add conffeti noise and prob. erasing = 1 and stronger perlin noise
-using only model3 since it was the one who get the best results until now

In [ ]:
train_set_11 = train_set
train_set_11 = train_set_11.concatenate(train_set.map(process_confetti_noise, num_parallel_calls = AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(process_perspective,num_parallel_calls=AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(process_crop,num_parallel_calls=AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(process_erasing,num_parallel_calls=AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_11 = train_set_11.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_11 = train_set_11.cache()
train_set_11 = train_set_11.map(process_image_testing, num_parallel_calls = AUTOTUNE)
train_set_11 = train_set_11.shuffle(buffer_size=30000)
train_set_11 = train_set_11.batch(batch_size=BATCH_SIZE)
train_set_11 = train_set_11.prefetch(buffer_size = AUTOTUNE)

val_set_11 = val_set
val_set_11 = val_set_11.cache()
val_set_11 = val_set_11.batch(batch_size = BATCH_SIZE)
val_set_11 = val_set_11.prefetch(buffer_size = AUTOTUNE)

test_set_11 = test_set
test_set_11 = test_set_11.batch(batch_size = BATCH_SIZE)

# Teste 19: model 3 dataset 11 -> Accuracy: 99.22

In [ ]:
model_3_11 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_3_11.load_weights('./cp.ckpt')

In [ ]:
file_path_3_11, callbacks_3_11 = prepareCallbacks('best_3_11')
history_3_11 = model_3_11.fit(train_set_11,
                              epochs = 50,
                              validation_data = val_set_11,
                              callbacks = callbacks_3_11,
                              verbose=2)

In [ ]:
eval_3_11 = model_3_11.evaluate(test_set_11, verbose=2)
print(eval_3_11)

val_3_11 = model_3_11.evaluate(val_set_11, verbose=2)

In [ ]:
for images, labels in test_set_11.take(-1):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_3_11.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 12: concat all pre-processing
-using only model3 since it was the one who get the best results until now

In [ ]:
train_set_12 = train_set
train_set_12 = train_set_12.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_perspective,num_parallel_calls=AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_crop,num_parallel_calls=AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_erasing,num_parallel_calls=AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_shear,num_parallel_calls=AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_hue,num_parallel_calls=AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_saturation, num_parallel_calls = AUTOTUNE))
train_set_12 = train_set_12.concatenate(train_set.map(process_brightness, num_parallel_calls = AUTOTUNE))
train_set_12 = train_set_12.cache()
train_set_12 = train_set_12.shuffle(buffer_size=50000)
train_set_12 = train_set_12.batch(batch_size=BATCH_SIZE)
train_set_12 = train_set_12.prefetch(buffer_size = AUTOTUNE)

val_set_12 = val_set
val_set_12 = val_set_12.cache()
val_set_12 = val_set_12.batch(batch_size = BATCH_SIZE)
val_set_12 = val_set_12.prefetch(buffer_size = AUTOTUNE)

test_set_12 = test_set
test_set_12 = test_set_12.batch(batch_size = BATCH_SIZE)

# Teste 20: model 3 dataset 12

In [ ]:
model_3_12 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_3_12.load_weights('./cp.ckpt')

In [ ]:
file_path_3_12, callbacks_3_12 = prepareCallbacks('best_3_12')
history_3_12 = model_3_12.fit(train_set_12,
                              epochs = 50,
                              validation_data = val_set_12,
                              callbacks = callbacks_3_12,
                              verbose=2)

In [ ]:
eval_3_12 = model_3_12.evaluate(test_set_12, verbose=2)
print(eval_3_12)

val_3_12 = model_3_12.evaluate(val_set_12, verbose=2)

In [ ]:
for images, labels in test_set_12.take(-1):  # take 10 batches of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = model_3_12.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Dataset 13: add random vertical and horizontal lines

In [ ]:
train_set_13 = train_set
train_set_13 = train_set_13.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_perspective,num_parallel_calls=AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_erasing,num_parallel_calls=AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_crop, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_vlines, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.concatenate(train_set.map(process_hlines, num_parallel_calls = AUTOTUNE))
train_set_13 = train_set_13.cache()
train_set_13 = train_set_13.map(process_shear, num_parallel_calls = AUTOTUNE)
train_set_13 = train_set_13.map(process_hue, num_parallel_calls = AUTOTUNE)
train_set_13 = train_set_13.map(process_saturation, num_parallel_calls = AUTOTUNE)
train_set_13 = train_set_13.map(process_brightness, num_parallel_calls = AUTOTUNE)
train_set_13 = train_set_13.shuffle(buffer_size=50000)
train_set_13 = train_set_13.batch(batch_size=BATCH_SIZE)
train_set_13 = train_set_13.prefetch(buffer_size = AUTOTUNE)

val_set_13 = val_set
val_set_13 = val_set_13.cache()
val_set_13 = val_set_13.batch(batch_size = BATCH_SIZE)
val_set_13 = val_set_13.prefetch(buffer_size = AUTOTUNE)

test_set_13 = test_set
test_set_13 = test_set_13.batch(batch_size = BATCH_SIZE)

# Teste 21: model 3 dataset 13 -> Accuracy: 99.38

In [ ]:
model_3_13 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_3_13.load_weights('./cp.ckpt')

In [ ]:
file_path_3_13, callbacks_3_13 = prepareCallbacks('best_3_13')
history_3_13 = model_3_13.fit(train_set_13,
                              epochs = 50,
                              validation_data = val_set_13,
                              callbacks = callbacks_3_13,
                              verbose=2)

In [ ]:
eval_3_13 = model_3_13.evaluate(test_set_13, verbose=2)
print(eval_3_13)

val_3_13 = model_3_13.evaluate(val_set_13, verbose=2)

# Dataset 14: different combinations

In [ ]:
train_set_14 = train_set
train_set_14 = train_set_14.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_14 = train_set_14.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_14 = train_set_14.concatenate(train_set.map(process_perspective,num_parallel_calls=AUTOTUNE))
train_set_14 = train_set_14.concatenate(train_set.map(process_erasing,num_parallel_calls=AUTOTUNE))
train_set_14 = train_set_14.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_14 = train_set_14.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_14 = train_set_14.concatenate(train_set.map(process_shear, num_parallel_calls = AUTOTUNE))
train_set_14 = train_set_14.cache()
train_set_14 = train_set_14.map(process_hue, num_parallel_calls = AUTOTUNE)
train_set_14 = train_set_14.map(process_saturation, num_parallel_calls = AUTOTUNE)
train_set_14 = train_set_14.map(process_brightness, num_parallel_calls = AUTOTUNE)
train_set_14 = train_set_14.shuffle(buffer_size=30000)
train_set_14 = train_set_14.batch(batch_size=BATCH_SIZE)
train_set_14 = train_set_14.prefetch(buffer_size = AUTOTUNE)

val_set_14 = val_set
val_set_14 = val_set_14.cache()
val_set_14 = val_set_14.batch(batch_size = BATCH_SIZE)
val_set_14 = val_set_14.prefetch(buffer_size = AUTOTUNE)

test_set_14 = test_set
test_set_14 = test_set_14.batch(batch_size = BATCH_SIZE)

# Teste 22: model 3 dataset 14 -> Accuracy: 99.26

In [ ]:
model_3_14 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_3_14.load_weights('./cp.ckpt')

In [ ]:
file_path_3_14, callbacks_3_14 = prepareCallbacks('best_3_14')
history_3_14 = model_3_14.fit(train_set_14,
                              epochs = 50,
                              validation_data = val_set_14,
                              callbacks = callbacks_3_14,
                              verbose=2)

In [ ]:
eval_3_14 = model_3_14.evaluate(test_set_14, verbose=2)
print(eval_3_14)

val_3_14 = model_3_14.evaluate(val_set_14, verbose=2)

# Dataset 15: different combinations

In [ ]:
train_set_15 = train_set
train_set_15 = train_set_15.concatenate(train_set.map(apply_perlin, num_parallel_calls = AUTOTUNE))
train_set_15 = train_set_15.concatenate(train_set.map(apply_blur, num_parallel_calls = AUTOTUNE))
train_set_15 = train_set_15.concatenate(train_set.map(process_perspective,num_parallel_calls=AUTOTUNE))
train_set_15 = train_set_15.concatenate(train_set.map(process_erasing,num_parallel_calls=AUTOTUNE))
train_set_15 = train_set_15.concatenate(train_set.map(process_translate, num_parallel_calls = AUTOTUNE))
train_set_15 = train_set_15.concatenate(train_set.map(process_rotation, num_parallel_calls = AUTOTUNE))
train_set_15 = train_set_15.cache()
train_set_15 = train_set_15.map(process_shear, num_parallel_calls = AUTOTUNE)
train_set_15 = train_set_15.map(process_hue, num_parallel_calls = AUTOTUNE)
train_set_15 = train_set_15.map(process_saturation, num_parallel_calls = AUTOTUNE)
train_set_15 = train_set_15.map(process_brightness, num_parallel_calls = AUTOTUNE)
train_set_15 = train_set_15.shuffle(buffer_size=30000)
train_set_15 = train_set_15.batch(batch_size=BATCH_SIZE)
train_set_15 = train_set_15.prefetch(buffer_size = AUTOTUNE)

val_set_15 = val_set
val_set_15 = val_set_15.cache()
val_set_15 = val_set_15.batch(batch_size = BATCH_SIZE)
val_set_15 = val_set_15.prefetch(buffer_size = AUTOTUNE)

test_set_15 = test_set
test_set_15 = test_set_15.batch(batch_size = BATCH_SIZE)

# Teste 23: model 3 dataset 15 -> Accuracy 99.16

In [ ]:
model_3_15 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_3_15.load_weights('./cp.ckpt')

In [ ]:
file_path_3_15, callbacks_3_15 = prepareCallbacks('best_3_15')
history_3_15 = model_3_15.fit(train_set_15,
                              epochs = 50,
                              validation_data = val_set_15,
                              callbacks = callbacks_3_15,
                              verbose=2)

In [ ]:
eval_3_15 = model_3_15.evaluate(test_set_15, verbose=2)
print(eval_3_15)

val_3_15 = model_3_15.evaluate(val_set_15, verbose=2)

# Ensembles

In [ ]:
# models
model_1 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_2 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_3 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_4 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)
model_5 = model_III(CLASS_COUNT, IMAGE_SIZE, 3)

# load model weights
model_1.load_weights('./model_1/cp.ckpt') # model_3_10 - 99.44
model_2.load_weights('./model_2/cp.ckpt') # model_3_11 - 99.22
model_3.load_weights('./model_3/cp.ckpt') # model_3_13 - 99.38
model_4.load_weights('./model_4/cp.ckpt') # model_3_14 - 99.26
model_5.load_weights('./model_5/cp.ckpt') # model_3_15 - 99.16

# get list of models
models = [model_1, model_2, model_3, model_4, model_5]

input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='input') # input layer

# get output for each model input
outputs = [model(input) for model in models]

# Concatenation Ensemble

In [ ]:
# contenate the ouputs
x = Concatenate()(outputs)

# add further layers
x = Dropout(0.5)(x)
output = Dense(CLASS_COUNT, activation='softmax', name='output')(x) # output layer

# create concatenated model
conc_model = Model(input, output, name='Concatenated_Model')
opt = Adam(learning_rate=0.0001)
conc_model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
# conc_model.load_weights('./drive/MyDrive/cp.ckpt')

In [ ]:
plot_model(conc_model)

In [ ]:
file_path_conc_model, callbacks_conc_model = prepareCallbacks('conc_model')

history_conc_model = conc_model.fit(train_set_10,
                                    epochs = 60,
                                    validation_data = val_set_10,
                                    callbacks = callbacks_conc_model,
                                    verbose=2)

In [ ]:
eval_conc_model = conc_model.evaluate(test_set_10, verbose=2)
print(eval_conc_model)

val_conc_model = conc_model.evaluate(val_set_10, verbose=2)

In [ ]:
for images, labels in test_set_10.take(-1):
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = conc_model.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Average Ensemble

In [ ]:
# take average of the outputs
output = Average()(outputs) # output layer

# create average ensembled model
avg_model = Model(input, output)
opt = Adam(learning_rate=0.0001)
avg_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
avg_model.load_weights('./drive/MyDrive/cp.ckpt')

In [ ]:
plot_model(avg_model)

In [ ]:
file_path_avg_model, callbacks_avg_model = prepareCallbacks('avg_model')

history_avg_model = avg_model.fit(train_set_10,
                                  epochs = 60,
                                  validation_data = val_set_10,
                                  callbacks = callbacks_avg_model,
                                  verbose=2)

In [ ]:
eval_avg_model = avg_model.evaluate(test_set_10, verbose=2)
print(eval_avg_model)

val_avg_model = avg_model.evaluate(val_set_10, verbose=2)

In [ ]:
for images, labels in test_set_10.take(-1):
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = avg_model.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)

# Weighted Average Ensemble

In [ ]:
# function for setting weights
def weight_init(shape =(1,1,5), weights=[0.3,0.15,0.28,0.17,0.1], dtype=tf.float32):
    return tf.constant(np.array(weights).reshape(shape), dtype=dtype)


# custom weighted average layer
class WeightedAverage(Layer):

    def __init__(self):
        super(WeightedAverage, self).__init__()

    def build(self, input_shape):

        self.W = self.add_weight(name="weight_layer",
                    shape=(1,1,len(input_shape)),
                    initializer=weight_init,
                    dtype=tf.float32,
                    trainable=True)

    def call(self, inputs):

        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs)
        weights = tf.nn.softmax(self.W, axis=-1)

        return tf.reduce_mean(weights*inputs, axis=-1)

In [ ]:
# take weighted average of the outputs
output = WeightedAverage()(outputs) # output layer

# create average ensembled model
weighted_avg_model = Model(input, output)
opt = Adam(learning_rate=0.0001)
weighted_avg_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weighted_avg_model.load_weights('./drive/MyDrive/cp.ckpt')

In [ ]:
plot_model(weighted_avg_model)

In [ ]:
file_path_weighted_avg_model, callbacks_weighted_avg_model = prepareCallbacks('weighted_avg_model')

history_weighted_avg_model = weighted_avg_model.fit(train_set_10,
                                                    epochs = 60,
                                                    validation_data = val_set_10,
                                                    callbacks = callbacks_weighted_avg_model,
                                                    verbose=2)

In [ ]:
eval_weighted_avg_model = weighted_avg_model.evaluate(test_set_10, verbose=2)
print(eval_weighted_avg_model)

val_weighted_avg_model = weighted_avg_model.evaluate(val_set_10, verbose=2)

In [ ]:
for images, labels in test_set_10.take(-1):
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

    preds = weighted_avg_model.predict(numpy_images)
    show_misclassified(preds, numpy_labels, numpy_images, 5, 3)